In [1]:
!pip install --upgrade mxnet
!pip install autogluon

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import autogluon as ag
from autogluon.tabular import TabularPredictor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 31.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
  Obtaining dependency information for autogluon from https://files.pythonhosted.org/packages/ce/7b/db15f5d33bb0bcdcaead1740ccb7018bc560a9fde14edd177c62f1051844/autogluon-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.core[all]==1.0.0 from https://files.pythonhosted.org/packages/d6/0b/d2d017b328a2101348c2a07752f57ae6f4c47bbb67e5527b10823f1db55b/autogluon.core-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.features==1.0.0 from https://files.pythonhosted.org/packages/a5/21/490e0a4310c7aaa22961c5885e2e03f0a6e8097d1d4af0f6c9b22d8fba50/autogluon.features-1.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for autogluon.tabular[all]==1.0.0 from https://files.py

In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s4e1/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s4e1/test.csv')
sample=pd.read_csv('/kaggle/input/playground-series-s4e1/sample_submission.csv')
df_all=pd.concat([train, test], sort=True).reset_index(drop=True)

In [3]:
print(df_all.info())
df_all.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275057 entries, 0 to 275056
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Age              275057 non-null  float64
 1   Balance          275057 non-null  float64
 2   CreditScore      275057 non-null  int64  
 3   CustomerId       275057 non-null  int64  
 4   EstimatedSalary  275057 non-null  float64
 5   Exited           165034 non-null  float64
 6   Gender           275057 non-null  object 
 7   Geography        275057 non-null  object 
 8   HasCrCard        275057 non-null  float64
 9   IsActiveMember   275057 non-null  float64
 10  NumOfProducts    275057 non-null  int64  
 11  Surname          275057 non-null  object 
 12  Tenure           275057 non-null  int64  
 13  id               275057 non-null  int64  
dtypes: float64(6), int64(5), object(3)
memory usage: 29.4+ MB
None


,Age,Balance,CreditScore,CustomerId,EstimatedSalary,Exited,Gender,Geography,HasCrCard,IsActiveMember,NumOfProducts,Surname,Tenure,id
0,33.0,0.0,668,15674932,181449.97,0.0,Male,France,1.0,0.0,2,Okwudilichukwu,3,0
1,33.0,0.0,627,15749177,49503.50,0.0,Male,France,1.0,1.0,2,Okwudiliolisa,1,1


In [4]:
df_all.describe()

,Age,Balance,CreditScore,CustomerId,EstimatedSalary,Exited,HasCrCard,IsActiveMember,NumOfProducts,Tenure,id
count,275057.000000,275057.000000,275057.000000,2.750570e+05,275057.000000,165034.000000,275057.000000,275057.000000,275057.000000,275057.000000,275057.000000
mean,38.124415,55420.296450,656.484939,1.569204e+07,112470.952558,0.211599,0.753589,0.496755,1.554002,5.010867,137528.000000
std,8.864927,62805.933171,80.188100,7.151271e+04,50286.608747,0.408443,0.430921,0.499990,0.546178,2.806173,79402.260834
min,18.000000,0.000000,350.000000,1.556570e+07,11.580000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,32.000000,0.000000,597.000000,1.563300e+07,74580.800000,0.000000,1.000000,0.000000,1.000000,3.000000,68764.000000
50%,37.000000,0.000000,659.000000,1.569017e+07,117848.090000,0.000000,1.000000,0.000000,2.000000,5.000000,137528.000000
75%,42.000000,120037.960000,710.000000,1.575688e+07,154871.130000,0.000000,1.000000,1.000000,2.000000,7.000000,206292.000000
max,92.000000,250898.090000,850.000000,1.581569e+07,199992.480000,1.000000,1.000000,1.000000,4.000000,10.000000,275056.000000


In [5]:
#Age group
df_all['Age_label']=0
df_all.loc[(df_all['Age']<20),'Age_label']=1
df_all.loc[(df_all['Age']>=20)&(df_all['Age']<30),'Age_label']=2
df_all.loc[(df_all['Age']>=30)&(df_all['Age']<40),'Age_label']=3
df_all.loc[(df_all['Age']>=40),'Age_label']=4

In [6]:
#Balance group
df_all['Balance_label']=0
df_all.loc[(df_all['Balance']>=50000)&(df_all['Balance']<100000),'Balance_label']=1
df_all.loc[(df_all['Balance']>=100000)&(df_all['Balance']<150000),'Balance_label']=2
df_all.loc[(df_all['Balance']>=150000)&(df_all['Balance']<200000),'Balance_label']=3
df_all.loc[(df_all['Balance']>=200000),'Balance_label']=4

In [7]:
#CreditScore group
df_all['CreditScore_label']=0
df_all.loc[(df_all['CreditScore']>=500)&(df_all['CreditScore']<600),'CreditScore_label']=1
df_all.loc[(df_all['CreditScore']>=600)&(df_all['CreditScore']<700),'CreditScore_label']=2
df_all.loc[(df_all['CreditScore']>=700),'CreditScore_label']=3

In [8]:
#EstimatedSalary group
df_all['EstimatedSalary_label']=0
df_all.loc[(df_all['EstimatedSalary']>=50000)&(df_all['EstimatedSalary']<100000),'EstimatedSalary_label']=1
df_all.loc[(df_all['EstimatedSalary']>=100000)&(df_all['EstimatedSalary']<150000),'EstimatedSalary_label']=2
df_all.loc[(df_all['EstimatedSalary']>=150000),'EstimatedSalary_label']=3

In [9]:
#mean Age
Gender_Age=df_all.groupby('Gender')['Age'].mean()
Geography_Age=df_all.groupby('Geography')['Age'].mean()
HasCrCard_Age=df_all.groupby('HasCrCard')['Age'].mean()
IsActiveMember_Age=df_all.groupby('IsActiveMember')['Age'].mean()
NumOfProducts_Age=df_all.groupby('NumOfProducts')['Age'].mean()
Tenure_Age=df_all.groupby('Tenure')['Age'].mean()

df_all=pd.merge(df_all,Gender_Age.rename('Gender_Age'),on=['Gender'],how='left')
df_all=pd.merge(df_all,Geography_Age.rename('Geography_Age'),on=['Geography'],how='left')
df_all=pd.merge(df_all,HasCrCard_Age.rename('HasCrCard_Age'),on=['HasCrCard'],how='left')
df_all=pd.merge(df_all,IsActiveMember_Age.rename('IsActiveMember_Age'),on=['IsActiveMember'],how='left')
df_all=pd.merge(df_all,NumOfProducts_Age.rename('NumOfProducts_Age'),on=['NumOfProducts'],how='left')
df_all=pd.merge(df_all,Tenure_Age.rename('Tenure_Age'),on=['Tenure'],how='left')

In [10]:
#mean Balance
Gender_Balance=df_all.groupby('Gender')['Balance'].mean()
Geography_Balance=df_all.groupby('Geography')['Balance'].mean()
HasCrCard_Balance=df_all.groupby('HasCrCard')['Balance'].mean()
IsActiveMember_Balance=df_all.groupby('IsActiveMember')['Balance'].mean()
NumOfProducts_Balance=df_all.groupby('NumOfProducts')['Balance'].mean()
Tenure_Balance=df_all.groupby('Tenure')['Balance'].mean()

df_all=pd.merge(df_all,Gender_Balance.rename('Gender_Balance'),on=['Gender'],how='left')
df_all=pd.merge(df_all,Geography_Balance.rename('Geography_Balance'),on=['Geography'],how='left')
df_all=pd.merge(df_all,HasCrCard_Balance.rename('HasCrCard_Balance'),on=['HasCrCard'],how='left')
df_all=pd.merge(df_all,IsActiveMember_Balance.rename('IsActiveMember_Balance'),on=['IsActiveMember'],how='left')
df_all=pd.merge(df_all,NumOfProducts_Balance.rename('NumOfProducts_Balance'),on=['NumOfProducts'],how='left')
df_all=pd.merge(df_all,Tenure_Balance.rename('Tenure_Balance'),on=['Tenure'],how='left')

In [11]:
#mean CreditScore 
Gender_CreditScore=df_all.groupby('Gender')['CreditScore'].mean()
Geography_CreditScore=df_all.groupby('Geography')['CreditScore'].mean()
HasCrCard_CreditScore=df_all.groupby('HasCrCard')['CreditScore'].mean()
IsActiveMember_CreditScore=df_all.groupby('IsActiveMember')['CreditScore'].mean()
NumOfProducts_CreditScore=df_all.groupby('NumOfProducts')['CreditScore'].mean()
Tenure_CreditScore=df_all.groupby('Tenure')['CreditScore'].mean()

df_all=pd.merge(df_all,Gender_CreditScore.rename('Gender_CreditScore'),on=['Gender'],how='left')
df_all=pd.merge(df_all,Geography_CreditScore.rename('Geography_CreditScore'),on=['Geography'],how='left')
df_all=pd.merge(df_all,HasCrCard_CreditScore.rename('HasCrCard_CreditScore'),on=['HasCrCard'],how='left')
df_all=pd.merge(df_all,IsActiveMember_CreditScore.rename('IsActiveMember_CreditScore'),on=['IsActiveMember'],how='left')
df_all=pd.merge(df_all,NumOfProducts_CreditScore.rename('NumOfProducts_CreditScore'),on=['NumOfProducts'],how='left')
df_all=pd.merge(df_all,Tenure_CreditScore.rename('Tenure_CreditScore'),on=['Tenure'],how='left')

In [12]:
#mean EstimatedSalary
Gender_EstimatedSalary=df_all.groupby('Gender')['EstimatedSalary'].mean()
Geography_EstimatedSalary=df_all.groupby('Geography')['EstimatedSalary'].mean()
HasCrCard_EstimatedSalary=df_all.groupby('HasCrCard')['EstimatedSalary'].mean()
IsActiveMember_EstimatedSalary=df_all.groupby('IsActiveMember')['EstimatedSalary'].mean()
NumOfProducts_EstimatedSalary=df_all.groupby('NumOfProducts')['EstimatedSalary'].mean()
Tenure_EstimatedSalary=df_all.groupby('Tenure')['EstimatedSalary'].mean()

df_all=pd.merge(df_all,Gender_EstimatedSalary.rename('Gender_EstimatedSalary'),on=['Gender'],how='left')
df_all=pd.merge(df_all,Geography_EstimatedSalary.rename('Geography_EstimatedSalary'),on=['Geography'],how='left')
df_all=pd.merge(df_all,HasCrCard_EstimatedSalary.rename('HasCrCard_EstimatedSalary'),on=['HasCrCard'],how='left')
df_all=pd.merge(df_all,IsActiveMember_EstimatedSalary.rename('IsActiveMember_EstimatedSalary'),on=['IsActiveMember'],how='left')
df_all=pd.merge(df_all,NumOfProducts_EstimatedSalary.rename('NumOfProducts_EstimatedSalary'),on=['NumOfProducts'],how='left')
df_all=pd.merge(df_all,Tenure_EstimatedSalary.rename('Tenure_EstimatedSalary'),on=['Tenure'],how='left')

In [13]:
#create money ratio
df_all['money_ratio']=(df_all['Balance']/df_all['EstimatedSalary'])

In [14]:
#feature convert
df_all['Surname'],uniques=pd.factorize(df_all['Surname'])
df_all=pd.get_dummies(df_all,columns=['Gender','Geography','HasCrCard','IsActiveMember','NumOfProducts','Age_label','Balance_label','CreditScore_label','EstimatedSalary_label'])

In [15]:
print(df_all.shape)
df_all.head()

(275057, 64)


,Age,Balance,CreditScore,CustomerId,EstimatedSalary,Exited,Surname,Tenure,id,Gender_Age,...,Balance_label_3,Balance_label_4,CreditScore_label_0,CreditScore_label_1,CreditScore_label_2,CreditScore_label_3,EstimatedSalary_label_0,EstimatedSalary_label_1,EstimatedSalary_label_2,EstimatedSalary_label_3
0,33.0,0.00,668,15674932,181449.97,0.0,0,3,0,37.628732,...,False,False,False,False,True,False,False,False,False,True
1,33.0,0.00,627,15749177,49503.50,0.0,1,1,1,37.628732,...,False,False,False,False,True,False,True,False,False,False
2,40.0,0.00,678,15694510,184866.69,0.0,2,10,2,37.628732,...,False,False,False,False,True,False,False,False,False,True
3,34.0,148882.54,581,15741417,84560.88,0.0,3,2,3,37.628732,...,False,False,False,True,False,False,False,True,False,False
4,33.0,0.00,716,15766172,15068.83,0.0,4,5,4,37.628732,...,False,False,False,False,False,True,True,False,False,False


In [16]:
#split data into train and test
df_all.drop(['id'], inplace=True, axis=1)
df_train=df_all.loc[df_all['Exited'].notnull()]
df_test=df_all.loc[df_all['Exited'].isnull()]
df_test.drop(['Exited'], inplace=True, axis=1)

/tmp/ipykernel_42/2797897533.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.drop(['Exited'], inplace=True, axis=1)


In [17]:
#prediction and submit
automl=TabularPredictor(label='Exited',problem_type='binary',eval_metric='roc_auc')
automl.fit(df_train, presets='best_quality')
automl.leaderboard().to_csv('leaderboard.csv')
automl.leaderboard()

No path specified. Models will be saved in: "AutogluonModels/ag-20240108_063923"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240108_063923/ds_sub_fit/sub_fit_ho.
2024-01-08 06:39:24,431	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning Au

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.893579,roc_auc,153.706450,2128.141599,0.031009,63.853953,3,True,24
1,WeightedEnsemble_L2,0.892489,roc_auc,32.605300,1317.898143,0.028531,37.193585,2,True,14
2,LightGBM_BAG_L2,0.892153,roc_auc,114.437194,1781.151028,2.384295,108.323389,2,True,16
3,CatBoost_BAG_L1,0.891876,roc_auc,0.312843,285.467822,0.312843,285.467822,1,True,7
4,LightGBMXT_BAG_L2,0.891867,roc_auc,115.853608,1800.268729,3.800709,127.441091,2,True,15
5,NeuralNetFastAI_BAG_L2,0.891854,roc_auc,115.597353,1800.379934,3.544453,127.552295,2,True,22
6,ExtraTreesGini_BAG_L2,0.891852,roc_auc,120.894641,1706.408251,8.841742,33.580612,2,True,20
7,XGBoost_BAG_L2,0.891819,roc_auc,113.376211,1695.416895,1.323312,22.589256,2,True,23
8,CatBoost_BAG_L2,0.891815,roc_auc,112.285306,1781.733575,0.232406,108.905937,2,True,19
9,ExtraTreesEntr_BAG_L2,0.891813,roc_auc,121.949323,1710.019860,9.896424,37.192222,2,True,21


In [18]:
predictions=automl.predict_proba(df_test)
predictions

INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsClassifier.predict_proba: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


,0,1
165034,0.984681,0.015319
165035,0.254264,0.745736
165036,0.983407,0.016593
165037,0.785636,0.214364
165038,0.586669,0.413331
...,...,...
275052,0.970217,0.029783
275053,0.869092,0.130909
275054,0.989929,0.010071
275055,0.834244,0.165756


In [19]:
predictions1=predictions[1]
predictions1

165034    0.015319
165035    0.745736
165036    0.016593
165037    0.214364
165038    0.413331
            ...   
275052    0.029783
275053    0.130909
275054    0.010071
275055    0.165756
275056    0.167245
Name: 1, Length: 110023, dtype: float64

In [20]:
output=pd.DataFrame({'id':sample['id'],'Exited':predictions1.values})
output.to_csv('submission.csv',index=False)

In [21]:
output

,id,Exited
0,165034,0.015319
1,165035,0.745736
2,165036,0.016593
3,165037,0.214364
4,165038,0.413331
...,...,...
110018,275052,0.029783
110019,275053,0.130909
110020,275054,0.010071
110021,275055,0.165756
